In [ ]:
# pip install torch transformers sentencepiece pandas numpy langchain


In [ ]:
# pip install llama-cpp-python


In [ ]:
#Running LLaMA 2 Locally (CPU)
from llama_cpp import Llama

model_path = "llama_model\llama-2-7b-chat.Q4_K_M.gguf"  # Download a GGUF quantized model
llm = Llama(model_path=model_path)

In [ ]:
import json
import re
from llama_cpp import Llama

def categorize_email(email_text, model_path):
    """
    Uses Llama 2 (locally) to categorize an email and extract key details.

    Args:
        email_text: The text content of the email.
        model_path: The path to the locally stored LLaMA 2 model.

    Returns:
        A JSON-formatted string with the categorized information.
    """
    prompt = f"""
You are an AI email analyzer for a bank. Categorize the email and extract key details.  
Your output should be a JSON object with the following keys:

- **request_type**: General category (e.g., "Information Request", "Support Request").  
- **sub_request_type**: Specific category (e.g., "Password Reset", "Order Status"). If not applicable, return "N/A".  
- **key_attributes**: A list of key details (e.g., ["Order Number: 12345", "Account ID: ABC1234"]). If none, return an empty list.  
- **main_intent**: A concise summary of why the email was written.  

Email:  
```{email_text}```  

Return JSON only, with no extra text before or after.
"""

    # Load LLaMA 2 locally
    llm = Llama(model_path=model_path)

    # Generate response
    response = llm(prompt, max_tokens=512)

    # Extract the text output
    response_text = response["choices"][0]["text"].strip()

    # Extract JSON using regex
    json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
    
    if json_match:
        try:
            result = json.loads(json_match.group(0))
            return json.dumps(result, indent=4)
        except json.JSONDecodeError:
            return json.dumps({"error": "Failed to parse JSON from the model response."}, indent=4)
    
    return json.dumps({"error": "No JSON response detected from the model."}, indent=4)




In [ ]:
email_samples = [
    # 1️⃣ Account Issue - Unrecognized Transaction
    """Subject: Urgent: Unrecognized Transaction on My Account

Dear Customer Support,

I recently noticed an unfamiliar transaction on my credit card statement. On **March 18, 2025**, an amount of **$245.99** was charged at "TechStore Inc." (Transaction ID: **TXN789456**). I do not recall making this purchase, and I am concerned that my card details might have been compromised.

Could you please investigate this charge and advise on the next steps? If necessary, I would like to dispute the transaction and request a card replacement.

Looking forward to your prompt response.

Best regards,  
**Michael Anderson**  
(Last 4 digits of card: **5678**)""" 
]

# Example usage with all sample emails:
model_path = "llama_model/llama-2-7b-chat.Q4_K_M.gguf"

for email in email_samples:
    print(categorize_email(email, model_path))
    print("-" * 80)
